# Project: Sampling


**Bogor, 5th July 2023**

---

In [1]:
# Load library
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt

c:\users\aiforesee\appdata\local\programs\python\python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\aiforesee\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.jpijnswnnan3ce6lli5fwsphut2vxmth.gfortran-win_amd64.dll
c:\users\aiforesee\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Set semua variable
N = 6

In [3]:
# masukkan data
# data diperoleh dari BPS

data = np.array([7546, 
                 4533, 
                 9782, 
                 6738, 
                 8855, 
                 8216
                ])

M_list = np.array([30,  
                   20, 
                   25,  
                   25,  
                   32, 
                   26
                  ])

---

# Jumlah Sample PSU

**Berapa banyak cluster yang dipilih jika target bound of error estimation total di set kurang dari $5000$**

Formula menghitung sample size:

$$
n = \cfrac{N \sigma_{c}^{2}}{N.D + \sigma_{c}^{2}}
$$

dengan
$$
D = \cfrac{B^{2}}{4 N^{2}}
$$

$$
\sigma_{c}^{2} = \cfrac{\sum_{i=1}^{N} (y_{i}-\hat{\mu}M_{i})^{2}}{N-1}
$$

$$
\hat{\mu}_{r} = r = \cfrac{\sum_{i=1}^{n} y_{i}}{\sum_{i=1}^{n} M_{i}}
$$

In [4]:
# hitung D
B = 5000
D = (B**2)/(4*(N**2))

# hitung rata-rata
mu_hat = np.sum(data) / np.sum(M_list)

# hitung sc_square
sc_square = 0
for i in range(N):
    sc_square += (data[i] - mu_hat*M_list[i])**2
sc_square *= (1/(N-1))

# hitung n
n = (N*sc_square) / ((N*D) + sc_square)
print(f"Jumlah sample PSU : {n:.0f}")

Jumlah sample PSU : 4


Sehingga, diperlukan psu sebanyak 4 cluster/kecamatan untuk mendapat error estimation total kurang dari 5000.

---

# Simulasi Cluster Sampling with Unequal Size  

**Population Total**

- Estimate population total

$$
\hat{\tau}_{r} = r M
$$

- dengan

$$
r = \cfrac{\sum_{i=1}^{n} y_{i}}{\sum_{i=1}^{n} M_{i}}
$$

- Approximate formula of variance

$$
\widehat{\text{var}}(\hat{\tau}_{r}) = \cfrac{N(N-n)}{n(n-1)} \sum_{i=1}^{n} (y_{i} - rM_{i})^{2}
$$

In [5]:
# import random 
from random import sample

# memilih 4 cluster dengan SRS
list_ = [0, 1, 2, 3, 4, 5] 
index_ = sample(list_, 4)
index_.sort()

index_

[1, 3, 4, 5]

In [6]:
# Set semua variable
N = 6
n = 4
M = np.sum(M_list)

# ambil data jumlah murid untuk 4 cluster terpilih
datas = [data[i] for i in range(N) if i in index_]

# ambil data jumlah ssu untuk 4 cluster terpilih 
M_lists = [M_list[i] for i in range(N) if i in index_]

datas, M_lists

([4533, 6738, 8855, 8216], [20, 25, 32, 26])

- Estimasi murid $\hat{\mu}_{r}$, dengan formula sebagai berikut

$$
\hat{\mu}_{r} = r = \cfrac{\sum_{i=1}^{n} y_{i}}{\sum_{i=1}^{n} M_{i}}
$$

- kemudian, estimate population total

$$
\hat{\tau}_{r} = r M
$$

In [7]:
# hitung rata-rata
mu_hat = np.sum(datas) / np.sum(M_lists)
print(f"Estimasi rata-rata murid\t: {mu_hat:.0f} murid")

# hitung tau
tau_hat = mu_hat*M
print(f"Estimasi Total murid\t\t: {tau_hat:.0f} murid")

Estimasi rata-rata murid	: 275 murid
Estimasi Total murid		: 43476 murid


- variance of total population estimator, $\widehat{\text{var}}(\hat{\tau}_{r})$, dengan formula sebagai berikut:

$$
\widehat{\text{var}}(\hat{\tau}_{r}) = N(N-n) \cfrac{s_{c}^{2}}{n}
$$

- sampled cluster variance ($s_{c}^{2}$) 

$$
s_{c}^{2} = \cfrac{1}{n-1} \sum_{i=1}^{n} (y_{i} - rM_{i})^{2}
$$

In [8]:
# hitung sampled cluster variance
sc_square = 0
for i in range(n):
    sc_square += (datas[i] - mu_hat*M_lists[i])**2
sc_square *= (1/(n-1))
print(f"The sampled cluster variance\t: ({np.sqrt(sc_square):.0f})^2")

# hitung variance of total population estimator
var_y_total_est = ((N*(N-n))/n)*sc_square
var_y_total_est
print(f"The variance of total pop. est.\t: ({np.sqrt(var_y_total_est):.0f})^2")

The sampled cluster variance	: (835)^2
The variance of total pop. est.	: (1446)^2


- 95% CI dapat dihitung dengan formula sebagai beirkut

$$
95 \% \text{CI} = \hat{\tau} \pm B
$$

- dengan Margin of error ($B$)
$$
B = z_{\alpha/2.} \sqrt{\widehat{\text{var}}(\hat{\tau})}
$$

In [9]:
# menghitung z_stat pada 95% CI
alpha = 0.05
z_stat = stats.norm.ppf(1 - alpha/2.)
print(f"z_stat              : {z_stat:.2f}")

# menghitung margin of error (B)
B = z_stat * np.sqrt(var_y_total_est)
print(f"margin of error (B) : {B:.0f} murid")

z_stat              : 1.96
margin of error (B) : 2834 murid


In [10]:
# menghitung CI
lb = tau_hat - B
ub = tau_hat + B

print(f"Dengan kepercayaan 95%, kita yakin bahwa total murid di Kota Bogor berada diantara {lb:.0f} and {ub:.0f} murid")

Dengan kepercayaan 95%, kita yakin bahwa total murid di Kota Bogor berada diantara 40642 and 46310 murid


In [11]:
# menghitung total murid sebenernya

total = np.sum(data)
print(f"Total murid sebenernya yaitu {total:.0f} murid")

Total murid sebenernya yaitu 45670 murid
